In [ ]:
# Fetching fear and greed index from alternative.me API
import requests

url = 'https://api.alternative.me/fng/?limit=0&format=json'
resp = requests.get(url)
resp.raise_for_status()
fng_data = resp.json()['data']

# Building a dataframe
fng = pd.DataFrame(fng_data)
# Turning timestamp to date
fng['timestamp'] = pd.to_datetime(fng['timestamp'].astype(int), unit='s').dt.date
# Turning value to int
fng['value'] = fng['value'].astype(int)

# Changing column names
fng = fng.rename(columns={
    'timestamp': 'date',
    'value': 'fear_greed_index',
    'value_classification': 'fear_greed_label'
})

# Dropping unnecassary column
fng = fng.drop(columns=['time_until_update'])

# Sorting based on date
fng = fng.sort_values(by='date').reset_index(drop=True)

fng.to_csv('outputs/fear_greed_index.csv', index=False)


In [ ]:
# Since there are multiple news per day, grouping data per day
df = pd.read_csv('outputs/binance_news_with_finbert.csv')
df['date'] = pd.to_datetime(df['date'], format='mixed')
df.set_index('date', inplace=True)

# Adding sentiment flags
df['positive'] = (df['finbert_sentiment'] == 'positive').astype(int)
df['negative'] = (df['finbert_sentiment'] == 'negative').astype(int)
df['neutral']  = (df['finbert_sentiment'] == 'neutral').astype(int)

# Aggregating per day
df_news_daily = df.groupby('date').agg({
    'positive': 'sum',
    'neutral': 'sum',
    'negative': 'sum',
    'headline': 'count',
    'finbert_sentiment': lambda x: x.value_counts().idxmax(),
    'finbert_score': 'mean'
}).rename(columns={
    'finbert_score': 'avg_sentiment',
    'headline': 'news_count'
}).reset_index()

df_news_daily.to_csv('outputs/binance_finbert_sentiments_daily.csv')

In [ ]:
# Combining fear and greed with news sentiments 

df_news = pd.read_csv('outputs/binance_finbert_sentiments_daily.csv')
df_fng = pd.read_csv('outputs/fear_greed_index.csv')

df_merged = df_news.merge(df_fng, how='outer')
df_merged.drop(columns='Unnamed: 0', inplace=True)

df_merged.to_csv('outputs/news_fag_final.csv')